In [1]:
# standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os # need for file enumeration
import pickle
from sklearn.model_selection import train_test_split, GridSearchCV


# CNN
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.applications import MobileNetV2,MobileNet,VGG19
from tensorflow.keras.applications import EfficientNetB7

from tensorflow.keras import utils,losses
import tensorflow as tf

# some more image preprocessing
from tensorflow.keras.layers import RandomBrightness, RandomContrast, RandomCrop, RandomFlip
from tensorflow.keras.layers import RandomHeight, RandomRotation, RandomTranslation
from tensorflow.keras.layers import RandomWidth, RandomZoom
import skimage
import cv2
from PIL.ExifTags import TAGS
from PIL import ExifTags
from PIL import Image

from os import listdir
from os.path import isfile, join


# pre-trained
from tensorflow.keras.applications import VGG19


# metrics
from sklearn.metrics import plot_confusion_matrix,ConfusionMatrixDisplay, confusion_matrix

# For reproducibility
np.random.seed(42)

/Users/glebradchenko/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
import pathlib
import skimage.io as io
import skimage.transform as tf
import torch

In [ ]:
from haroun import Data, Model, ConvPool
from haroun.augmentation import augmentation
from haroun.losses import rmse
def load_data():
    path =  "./data/real_and_fake_face/"
    images = []
    labels = []


    for directory in os.listdir(path):
        data_path = path + directory
        if directory!=".DS_Store":

            for im in os.listdir(data_path)[:]:
                image = io.imread(f"{data_path}/{im}")
                image = tf.resize(image, (64, 64))
                images.append(image)
                if directory == "training_fake":
                    labels.append("fake")
                elif directory == "training_real":
                    labels.append("real")
    

    images = np.array(images)
    labels = np.array(labels)

    # images, labels = augmentation(images, labels, flip_y=True, flip_x=True, brightness=True)

    return images, labels


In [ ]:
classes = {'real': 0, 'fake': 1}
data = Data(loader=load_data(), classes=classes)
data.shape()

In [ ]:
data.show()

In [ ]:
data.stat()

In [ ]:
# dtype = torch.float
device = torch.device("mps")
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data.dataset(split_size=0.05, shuffle=True, random_state=42,
             images_format=torch.float32, labels_format=torch.float32,
             permute=True, one_hot=True, device=device)

In [ ]:
class Network(torch.nn.Module):

    def __init__(self):
        super(Network, self).__init__()
        self.input_norm = torch.nn.BatchNorm2d(3, affine=False)
        self.layer1 = ConvPool(in_features=3, out_features=8)
        self.layer2 = ConvPool(in_features=8, out_features=16)
        self.layer3 = ConvPool(in_features=16, out_features=32)
        self.layer4 = ConvPool(in_features=32, out_features=64)
        self.layer5 = ConvPool(in_features=64, out_features=128)
        self.layer6 = ConvPool(in_features=128, out_features=256)
        
        

        self.net = torch.nn.Sequential(self.layer1, self.layer2, self.layer3, 
                                       self.layer4, self.layer5, self.layer6)
            
        
        self.fc1 = torch.nn.Linear(in_features=256, out_features=128)
        self.bn1 = torch.nn.BatchNorm1d(128)
        
        self.fc2 = torch.nn.Linear(in_features=128, out_features=32)
        self.bn2 = torch.nn.BatchNorm1d(32)

        self.fc3 = torch.nn.Linear(in_features=32, out_features=8)
        self.bn3 = torch.nn.BatchNorm1d(8)

        self.fc4 = torch.nn.Linear(in_features=8, out_features=2)


        self.lin = torch.nn.Sequential(self.fc1, self.bn1, self.fc2, self.bn2,
                                       self.fc3, self.bn3, self.fc4)  


    def forward(self, X: torch.Tensor) -> torch.Tensor:
        X = self.input_norm(X)
        X = self.net(X)
        X = X.reshape(X.size(0), -1)
        X = self.lin(X)
        X = torch.nn.functional.elu(X, alpha=1.0, inplace=False)
        return X

In [ ]:
net = Network()
AntiSpoofClassifier = Model(net, "adam", rmse, device)
AntiSpoofClassifier.train(train_data=(data.train_inputs, data.train_outputs),
                          val_data=(data.val_inputs, data.val_outputs),
                          epochs=200, patience=20, batch_size=100, learning_rate=1.0E-3)